In [ ]:
using JuMP
using CPLEX

In [ ]:
#matriz de distancias aleatorias
function instanciaAleatoria(n::Int64)
    l = rand(1:1000, n*n)
    for i in 1:n
        l[(i-1)*n+i]=0
    end
    return reshape(l,n,n)
end

In [ ]:
n=10
distances=instanciaAleatoria(n)

In [ ]:
#finds greedy solution starting from ciudadInicio
function greedy(distancias,n,ciudadInicio)
    yavisitados=zeros(Bool,n)
    distanciaAcumulada=0
    actual=ciudadInicio
    yavisitados[ciudadInicio]=true
    tour=[actual]
    #println("empiezo en ",actual)
    for ciudadesVisitadas in 2:n
        #buscar la ciudad mas cercana a actual no visitada 
        distanciaActual=10000000
        siguiente=0
        for i in 1:n
            if yavisitados[i]==false #no visitado
                if distanciaActual>=distancias[actual,i] #mas cercano?
                    distanciaActual=distancias[actual,i] # lo guardo
                    siguiente=i
                end
            end
        end
        # siguiente tiene el vertice mas cercano al actual
        distanciaAcumulada += distanciaActual
        actual=siguiente
        push!(tour, actual)
        yavisitados[actual]=true
        #println("paso a ",actual," con distancia acumulada ",distanciaAcumulada)
    end
    distanciaAcumulada += distancias[actual,1]
    #distanciaAcumulada = distanciaAcumulada + distancias[actual,1]
    return distanciaAcumulada, tour
end

In [ ]:
#helper for local search (takes permutation and finds length of this tour)
function evaluate(distancias,n,tour)
    distanciaAcumulada=0
    for i in 1:n-1
        distanciaAcumulada += distancias[tour[i],tour[i+1]]
    end
    distanciaAcumulada += distancias[tour[n],tour[1]]
    return distanciaAcumulada
end

In [ ]:
#simple local search. Neighborhood is defined as the removal and insertion of a vertex
function localsearch(distancias,n,tour,length)
    mejora=true
    while mejora
        mejora=false
        for i in 1:n
            base=filter(x->x!=i,tour) #removes i from route and store in base
            #println("base: ",base)
            for position in 1:n
                newRoute=deepcopy(base)
                insert!(newRoute,position,i)
                #println("nr: ",newRoute)
                newObj=evaluate(distancias,n,newRoute)
                if newObj<length
                    #println(" newObj: ",newObj," ",newRoute)
                    length=newObj
                    tour=deepcopy(newRoute)
                    mejora=true
                    break
                end
            end
        end            
    end
    return length,tour
end

In [ ]:
#finds n different starting solutions and applies lo9cal search to each of them
for i in 1:n
    distancia,tour=greedy(distances,n,i)
    print("initial length: ",distancia)
    distancia,tour=localsearch(distances,n,tour,distancia)
    println(" after local search: ",distancia)
end

In [ ]:
#helpers for exact model
# extractTour provides a subtour according to current ip solution (sol)
function extractTour!(n, initial,sol,inCircuit)
    tour = [initial]  # Start at initial city
    cur_city = initial
    numVisited=0
    while true
        # Look for outgoing arc
        for j = 1:n
            if sol[cur_city,j] >= 0.9 #value 1
                # Found next city
                cur_city=j
                push!(tour, j)
                inCircuit[j]=1
                # Mark as visited
                numVisited += 1
                break
            end
        end
        # If we have come back to initial city, stop
        if cur_city == initial
            break
        end
    end  # end while
    println("tour: ",tour," numVisited ",numVisited)
    return tour, numVisited, inCircuit
end

# reports solution according to current ip solution (sol)
function reportSolution(n, initial,sol)
    tour = [initial]  # Start at initial city
    cur_city = initial
    while true
        # Look for first arc out of current city
        for j = 1:n
            if sol[cur_city,j] >= 0.9 #value 1
                # Found next city
                cur_city=j
                push!(tour, j)
                # Mark as visited
                break
            end
        end
        # If we have come back to initial city, stop
        if cur_city == initial
            break
        end
    end  # end while
    return tour
end

In [ ]:
#builds model and adds lazy constraint

function solveATSP(distances, n::Int64)
    initialtime=time()
    m =Model(CPLEX.Optimizer) 
    set_parameter(m,"CPX_PARAM_SCRIND",0)
    
    @variable(m, x[1:n,1:n], Bin)

    @objective(m, Min, sum(distances[i,j]*x[i,j] for i in 1:n for j in 1:n))
    @constraint(m, outgoing[i=1:n],x[i,i] == 0)
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1) # $sum_{j=1}^n x_{ij}=1 \forall i$
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1) # $sum_{i=1}^n x_{ij}=1 \forall j$
    
    function lazyGenerator(cb)
        println("---- Inside subtour callback")
        inCircuit=fill(false,n)
        x_val = callback_value.(Ref(cb), x)
        for city in 1:n
            if inCircuit[city]==0 
                # sum arcs in subtour <= |cities| -1
                subtour, numVisited,inCircuit = extractTour!(n, city,x_val,inCircuit)
                if numVisited == n
                    # This "subtour" is actually all cities, so we are done
                    println("Solution visits all cities")
                    return
                end
                arcs_from_subtour = zero(AffExpr)
                for i =1:numVisited
                    arcs_from_subtour += x[subtour[i],subtour[i+1]]
                end
                MOI.submit(m, MOI.LazyConstraint(cb), @build_constraint(arcs_from_subtour <= numVisited-1))
            end
        end
    end
    MOI.set(m, MOI.LazyConstraintCallback(), lazyGenerator)
    optimize!(m)
    println("total time: ",time()-initialtime)
    solution = reportSolution(n, 1,JuMP.value.(x))
    println(solution," with length: ",getobjectivevalue(m))                    
end

In [ ]:
#solves model

solveATSP(distances,n)